## pytest

PyTest - модуль для тестирования кода

#### Немного о тестировании в целом:

**Интеграционное тестирование** - тестирование взаимодействия компонентов. Проверка на работоспособность всего проекта со всеми зависимостями.

-) Сложно, трудно автоматизировать, требует сторонние зависимости, долго

**Юнит-тестирование (модульное тестирование)** - проверка на корректность отдельных модулей. Тестирование конкретных функций и методов.

Иерархия тестирования:

**Manual tests** (высокий уровень)

**Acceptance tests**

**Integration tests**

**Unit tests** (низкий уровень)

Тесты представляют **контракт**, **спецификацией**. Какие входные параметры, какие выходные, что ожидается в процессе работы

**Разделяй и властвуй**. Тесты должны проверять одну вещь в один момент времени. Это упрощает как поддержку тестов, так и возможность отловить баги на самой ранней стадии

**TDD** (Test-driven development). Техника разработки ПО, которая основывается на повторении коротких циклов разработки: test -> code. Сначала пишутся тесты для желаемого поведения, затем - код, который должен проходить эти тесты.

При решении добавить новую функциональность для приложения, сначала пишутся тесты для того, что эта функциональность должна делать, а только затем код

Написание 'Hello world' для тестирования с **pytest**

### Упражнение 1:

Создайте файл *test_simple.py* и вставьте в него код, представленный ниже

In [2]:
import pytest

def setup_module(module):
    #init_something()
    pass

def teardown_module(module):
    #teardown_something()
    pass

def test_upper():
    assert 'foo'.upper() == 'FOO'
    
def test_isupper():
    assert 'FOO'.isupper()
    
def test_failed_upper():
    assert 'foo'.upper() == 'FOo'

In [4]:
# Запустите тесты командой:
!pytest -v test_simple.py
# pytest рекурсивно обходит все файлы в директории (+ вложенные) и запускает тесты в файлах test_*

============================= test session starts =============================
platform win32 -- Python 3.6.5, pytest-3.5.1, py-1.5.3, pluggy-0.6.0 -- c:\programdata\anaconda3\python.exe
cachedir: .pytest_cache
rootdir: D:\mofw, inifile:
plugins: remotedata-0.2.1, openfiles-0.3.0, doctestplus-0.1.3, arraydiff-0.2
collecting ... collected 3 items

test_simple.py::test_upper PASSED                                        [ 33%]
test_simple.py::test_isupper PASSED                                      [ 66%]
test_simple.py::test_failed_upper FAILED                                 [100%]

================================== FAILURES ===================================
______________________________ test_failed_upper ______________________________

    def test_failed_upper():
>       assert 'foo'.upper() == 'FOo'
E       AssertionError: assert 'FOO' == 'FOo'
E         - FOO
E         + FOo

test_simple.py:18: AssertionError
===================== 1 failed, 2 passed in 0.06 seconds ===========

Видим подробный отчет о тестировании со всей информацией по тестам (В командной строке будет подсветка синтаксиса)

pytest позволяет задать функции инициализации и деструкторы перед и после тестов

Запустите код, представленный ниже

In [ ]:
import pytest
 
@pytest.fixture(scope="module")
def resource_setup(request):
    print("\nconnect to db")
    db = {"Red":1,"Blue":2,"Green":3}
    def resource_teardown():
        print("\ndisconnect")
    request.addfinalizer(resource_teardown)
    return db

def test_db(resource_setup):
    for k in resource_setup.keys():
        print "color {0} has id {1}".format(k, resource_setup[k])

def test_red(resource_setup):
    assert resource_setup["Red"] == 1

def test_blue(resource_setup):
    assert resource_setup["Blue"] != 1

Предоставление возможности для нескольких тестов вынести открытие, закрытие базы данных в отдельную функцию

Пример проверки чисел Фибоначчи (более структурированно):

In [ ]:
def get_fibonacci_value(n, first_val, second_val):
    raise RuntimeError("Not implemented")

In [ ]:
import pytest    

class Case(object):
    def __init__(self, position: int, first_value: int,
                 second_value: int, expected: int):
        self.position = position
        self.expected = expected
        self.first_value = first_value
        self.second_value = second_value

    def __str__(self):
        return "test_{}".format(self.position)


TEST_CASES = [
    Case(position=1, first_value=0, second_value=1, expected=1)
    # Можно добавить дополнительные тесты
]


@pytest.mark.parametrize("test_case", TEST_CASES, ids=[str(t) for t in TEST_CASES])
def test_fibonacci(test_case: Case):
    combinations = get_fibonacci_value(
        test_case.position, test_case.first_value, test_case.second_value
    )
    assert combinations == test_case.expected


Тестирование методов класса:

In [ ]:
# cool.py
class SuperCool(object):
    def action(self, x):
        return x * x

In [ ]:
# test_cool.py
from cool import SuperCool

class TestSuperCool(object):
    def test_action(self):
        sc = SuperCool()
        assert sc.action(1) == 1

Создание объекта класса единожды для всех тестов:

In [ ]:
# test_cool.py
import pytest
from cool import SuperCool

class TestSuperCool(object):
    def setup(self):
        self.sc = SuperCool()

    def test_action(self):
        assert self.sc.action(1) == 1
    
    # parametrize разворачивает несколько тестов
    @pytest.mark.parametrize("element, expected", [(1, 1), (2, 4)])
    def test_action_with_parametrization(self, element, expected):
        assert self.sc.action(element) == expected
    
    # randomize
    @pytest.mark.randomize(num=int, min_num=3, max_num=1000, ncalls=5)
    def test_quickcheck(self, num):
        assert self.sc.action(num) == num**2

### Упражнение 2:

Напишите класс **Stack** и покройте его тестами (использовать parametrize и randomize)

Основные функции: push, pop, last, size

Должно быть 2 файла:
- stack.py
- test_stack.py

Больше информации: https://docs.pytest.org/en/latest/